In [1]:
from neuralforecast.core import NeuralForecast
from neuralforecast.models import TimesNet
from neuralforecast.losses.numpy import mae, mse
from neuralforecast.losses.pytorch import HuberLoss
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import torch
import cleandata
import sys
import numpy as np
import pandas as pd
import os

In [3]:
pwd = "C:/programming/MADDQN"
github_url = 'https://raw.githubusercontent.com/CodeBeckZero/MADDQN/main/input_data'
sys.path.append(pwd)

# Input Data Location, File Name, Stock name for labels
import_path = os.path.join(pwd, "input_data")

# Output Path Location for CSV export
export_path = os.path.join(pwd, "output_data")

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the current device
    device = torch.cuda.current_device()
    print(f"GPU Name: {torch.cuda.get_device_name(device)}")
else:
    device = 'cpu'
    print("CUDA (GPU support) is not available. PyTorch is running on CPU.")


stock_inputs ={'DJI':'^DJI_daily.csv',
               'NDAQ': '^IXIC_daily.csv',
               'SP500': '^SPX_daily.csv',
               'AAPL': 'AAPL_daily.csv',
               'AMZN': 'AMZN_daily.csv',
               'GOOGL': 'GOOGL_daily.csv',
               'MSFT': 'MSFT_daily.csv',
               'SINE': 'sine_wave_daily.csv',
               'FORD': 'F_daily.csv',
               'JNJ': 'JNJ_daily.csv',
               'NEE': 'NEE_daily.csv',
               'PFE': 'PFE_daily.csv',
               'TSLA': 'TSLA_daily.csv'}

# Blending Inputs
blend_name = 'USIDX'
blend_keys = ['NDAQ','DJI','SP500']
blend_col_keys = ['open','high','low', 'close', 'volume']

# Training Inputs
trn_keys = ['NDAQ','DJI','SP500']
training_range = (0,2500)
validation_range = (2500,2750)

# Testing Inputs
tst_keys = ['NDAQ']
testing_range = (2750,3000)

envs_to_create = set(blend_keys + trn_keys + tst_keys)

environments = {}
for name, file in stock_inputs.items():
    if name in envs_to_create:
        print(name)
        temp_df = cleandata.YAHOO_csv_input(file,github_url)
        temp_norm_df = cleandata.normalize_df_ohlcv_by_row_range(temp_df,training_range[0],training_range[1])
        temp_norm_df['date'] = temp_df['date']
        environments[name] = temp_norm_df

display(environments['NDAQ'].head())
display(environments['DJI'].head())
display(environments['SP500'].head())

dfs_to_blend = []

for key in blend_keys:
    dfs_to_blend.append(environments[key])

environments[blend_name] = cleandata.avg_datasets(dfs_to_blend,blend_col_keys)

display(environments['USIDX'].head())



CUDA (GPU support) is not available. PyTorch is running on CPU.
DJI
NDAQ
SP500


,open,high,low,close,volume,date
0,-0.717597,-0.711659,-0.730972,-0.723819,1.066433,2007-01-03
1,-0.722910,-0.706364,-0.713748,-0.696565,0.274391,2007-01-04
2,-0.703774,-0.720245,-0.707576,-0.713834,0.169486,2007-01-05
3,-0.712617,-0.719742,-0.707089,-0.710278,-0.200710,2007-01-08
4,-0.705404,-0.715930,-0.704896,-0.705209,0.369966,2007-01-09


,open,high,low,close,volume,date
0,-0.369748,-0.357582,-0.358433,-0.365693,1.592781,2007-01-03
1,-0.365221,-0.380883,-0.358751,-0.363643,0.882276,2007-01-04
2,-0.362931,-0.390971,-0.371495,-0.391109,0.633693,2007-01-05
3,-0.392190,-0.402368,-0.380788,-0.382645,0.511487,2007-01-08
4,-0.381303,-0.395535,-0.370249,-0.384934,0.529108,2007-01-09


,open,high,low,close,volume,date
0,-0.237904,-0.232293,-0.238372,-0.242264,-0.517929,2007-01-03
1,-0.241576,-0.251783,-0.236912,-0.237799,-0.874033,2007-01-04
2,-0.237109,-0.260783,-0.243780,-0.259946,-0.945354,2007-01-05
3,-0.260419,-0.269422,-0.248342,-0.251914,-1.076208,2007-01-08
4,-0.251228,-0.267803,-0.244626,-0.253787,-0.845591,2007-01-09


,open,high,low,close,volume,date
0,-0.441750,-0.433845,-0.442593,-0.443925,0.713762,2007-01-03
1,-0.443236,-0.446343,-0.436470,-0.432669,0.094211,2007-01-04
2,-0.434605,-0.457333,-0.440950,-0.454963,-0.047392,2007-01-05
3,-0.455075,-0.463844,-0.445407,-0.448279,-0.255144,2007-01-08
4,-0.445978,-0.459756,-0.439924,-0.447976,0.017828,2007-01-09


'        \n    \nALPHA = 0.1\nGAMMA = 0.9\n\n# Parameters\nbatch_len = 10 # Length of data in each patch\nprd_range = testing_range[1] - validation_range[0]\n\ndata = pd.read_csv(import_path + \'/\'+ stock_inputs[\'NDAQ\'])\ndata[\'Date\'] = pd.to_datetime(data[\'Date\'])\nvalue_names = [col for col in data.columns if col != \'date\']\ndf_by_value = {}\nfor value in value_names:\n    # Selecting columns \'date\' and \'value_column\' for the new DataFrame\n    new_df = data[[\'Date\', value]].copy()\n\n    # Renaming columns \'date\' to \'ds\' and \'value_column\' to \'y\'\n    new_df = new_df.rename(columns={\'Date\': \'ds\', value: \'y\'})\n\n    # Adding a \'unique_id\' column to the new DataFrame\n    new_df.insert(1,\'unique_id\',value)\n\n    # Store the updated DataFrame in the dictionary\n    df_by_value[value] = new_df\ndisplay(df_by_value["Volume"])\n\nenvironments = {}\n\nfor name, file in stock_inputs.items():\n    temp_df = cleandata.YAHOO_csv_input(file,import_path)\n    t

In [ ]:
ALPHA = 0.1
GAMMA = 0.9

# Parameters
batch_len = 10 # Length of data in each patch
prd_range = testing_range[1] - validation_range[0]


In [ ]:
model = TimesNet(h = prd_range, # Forecast horizon
                 input_size = batch_len, # Length of input window (lags)
                 hidden_size = 64, # Size of embedding for embedding and encoders,
                 dropout = 0.1, # Dropout for embeddings
                 conv_hidden_size = 5, # Channels for the inception block
                 top_k = 5, # Top num of periods from FFT considered
                 num_kernels = 13, # number of kernels for the inception block
                 encoder_layers = 3, # num of encoders
                 max_steps = 100, # of training steps
                 early_stop_patience_steps = -1, #early stoppage on validation
                 val_check_steps = 5 # Val check every X steps,
                 windows_batch_size = 64 # Number of windows in training epoch,
                 loss = HuberLoss,
                 learning_rate = 0.0008)


trn_df = pd.DataFrame(df_by_value['Close'][training_range[0]:training_range[1]])
test_df = pd.DataFrame(df_by_value['Close'][validation_range[0]:testing_range[1]])
nf = NeuralForecast(models=[model], freq='d')
nf.fit(df=trn_df)
forecast = nf.predict(df=test_df)


                  
                 
                  